# Consulta de API y Estructuración de Datos

Este notebook muestra cómo usar `requests` para consultar una API y `pandas` para estructurar y analizar los datos obtenidos.


In [1]:
import requests
import pandas as pd
import json
from datetime import datetime


## 1. Autenticación en la API

Realizamos el login para obtener el token de autenticación que usaremos en las siguientes solicitudes.


In [6]:
url_login = "http://localhost:8080/api/auth/login"

credenciales = {
    "identification": "1000747836",
    "password": "Admin2024!"
}

response = requests.post(url_login, json=credenciales)

if response.status_code == 200:
    data_response = response.json()
    token = data_response.get('token') or data_response.get('accessToken') or data_response.get('access_token')
    
    if token:
        print(f"Autenticación exitosa - Status Code: {response.status_code}")
        AUTH_TOKEN = token
        token_type = data_response.get('tokenType', 'Bearer') or 'Bearer'
        AUTH_TOKEN_TYPE = token_type
    else:
        print("Error: No se pudo obtener el token de la respuesta")
else:
    print(f"Error en la autenticación - Status Code: {response.status_code}")


Autenticación exitosa - Status Code: 200


In [7]:
def get_auth_headers():
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'{AUTH_TOKEN_TYPE} {AUTH_TOKEN}'
    }
    return headers


## 2. Consulta de documentos fiscales

Realizamos una petición al endpoint de documentos fiscales usando el token de autenticación.


In [8]:
url_fiscal_documents = "http://localhost:8080/api/fiscal-documents"
headers = get_auth_headers()

response = requests.get(url_fiscal_documents, headers=headers)

if response.status_code == 200:
    fiscal_documents_data = response.json()
    print(f"Petición exitosa - Status Code: {response.status_code} - Registros obtenidos: {len(fiscal_documents_data)}")
else:
    print(f"Error en la petición - Status Code: {response.status_code}")


Petición exitosa - Status Code: 200 - Registros obtenidos: 0


## 3. Crear DataFrame con los datos

Convertimos los datos JSON obtenidos de la API en un DataFrame de pandas para facilitar su análisis y manipulación.


In [5]:
if 'fiscal_documents_data' in globals() and fiscal_documents_data:
    df_fiscal_documents = pd.DataFrame(fiscal_documents_data)
    print(f"DataFrame creado - Registros: {len(df_fiscal_documents)}")
else:
    print("Error: No hay datos disponibles para crear el DataFrame")


DataFrame creado - Registros: 24


## 4. Filtrar DataFrame por group_info

Filtramos el DataFrame para obtener solo los registros donde group_info sea igual a "Emitido".


In [ ]:
if 'df_fiscal_documents' in globals():
    # Filtrar el dataframe por group_info == "Emitido"
    df_fiscal_documents = df_fiscal_documents[df_fiscal_documents['groupInfo'] == 'Emitido']
    print(f"DataFrame filtrado - Registros con group_info='Emitido': {len(df_fiscal_documents)}")
    print(f"\nPrimeras filas del DataFrame filtrado:")
    print(df_fiscal_documents.head())
else:
    print("Error: No existe el DataFrame df_fiscal_documents")


## 5. Suma de la columna total

Calculamos la suma total de la columna "total" después del filtro aplicado.


In [ ]:
if 'df_fiscal_documents' in globals() and not df_fiscal_documents.empty:

    if 'total' in df_fiscal_documents.columns:
        suma_total = df_fiscal_documents['total'].sum()
        print(f"Suma total de la columna 'total': {suma_total:,.2f}")
        
        print(f"\nCantidad de registros: {len(df_fiscal_documents)}")
        print(f"Valor mínimo: {df_fiscal_documents['total'].min():,.2f}")
        print(f"Valor máximo: {df_fiscal_documents['total'].max():,.2f}")
        print(f"Valor promedio: {df_fiscal_documents['total'].mean():,.2f}")
    else:
        print("Error: No existe la columna 'total' en el DataFrame")
        print(f"Columnas disponibles: {list(df_fiscal_documents.columns)}")
else:
    print("Error: No existe el DataFrame df_fiscal_documents o está vacío")


Suma total de la columna 'total': 83,380,493.44

Cantidad de registros: 24
Valor mínimo: 35,000.00
Valor máximo: 23,874,280.65
Valor promedio: 3,474,187.23
